In [1]:
from urllib.request import Request, urlopen, HTTPError
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import pandas as pd
import csv

keyword = ['데이터 사이언티스트', '데이터애널리스트', '데이터 엔지니어']
page = 1
driver = webdriver.Chrome()

Title = []
Company = []
Link = []
Location = []
label = []
StackTool = []

# url, 공고 이름, 기업명, 기업 주소를 찾는 크롤링코드
for search_keyword in keyword:
    driver.get(f'https://www.saramin.co.kr/zf_user/search?searchType=search&searchword={search_keyword}&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage={page}&recruitSort=relation&recruitPageCount=100&inner_com_type=scale001&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=y')
    ct_result = driver.find_element(By.XPATH, '//*[@id="recruit_info"]/div[1]/span').text
    total_result = re.sub(r'[^0-9]', '', ct_result)
    page_ct = int(int(total_result)/100)

    for pages in range(1, page_ct+2):
        driver.get(f'https://www.saramin.co.kr/zf_user/search?searchType=search&searchword={search_keyword}&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage={page}&recruitSort=relation&recruitPageCount=100&inner_com_type=scale001&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=y')
        driver.implicitly_wait(time_to_wait=60)

        div_title = driver.find_elements(By.CLASS_NAME, "job_tit")
        div_company = driver.find_elements(By.CLASS_NAME, "corp_name")
        div_stacktool = driver.find_elements(By.CLASS_NAME, "job_sector")
        div_location = driver.find_elements(By.CLASS_NAME, "job_condition")
        
        try:
            for item in div_title:
                tmp_url = item.find_element(By.TAG_NAME, "a").get_attribute('href')
                Link.append(tmp_url)
        except Exception as e:
            Link.append("NaN")
            pass
        
        try:
            for item in div_title:
                tmp_title = item.find_element(By.TAG_NAME, "a").text
                Title.append(tmp_title)
        except Exception as e:
            Title.append("NaN")
            pass
        
        try:
            for item in div_company:
                tmp_cn = item.find_element(By.TAG_NAME, "a").text
                Company.append(tmp_cn)
                label.append(search_keyword)
        except Exception as e:
            Company.append("NaN")
            label.append("NaN")
            pass
        
        try:
            for item in div_stacktool:
                tmp_stacktool = []
                tmp_stack = item.find_elements(By.TAG_NAME, "a")
                for i in tmp_stack:
                    stack = i.text
                    tmp_stacktool.append(stack)
                StackTool.append(tmp_stacktool)
        except Exception as e:
            StackTool.append("NaN")
            pass
        
        try:
            for loc in div_location:
                tmp_location = loc.find_element(By.TAG_NAME, "span").text
                Location.append(tmp_location)        
        except Exception as e:
            Location.append("NaN")
            pass
        

In [2]:
driver = webdriver.Chrome("chromedriver")
#기업 주소, 공고내용 전체 크롤링 코드
Content=[]

for url in Link:
    driver.get(url)
    driver.implicitly_wait(time_to_wait=60)

    try:
        driver.switch_to.frame("iframe_content_0")
        contents = driver.find_element(By.CLASS_NAME, "user_content").text
        Content.append(contents)
        
    except:
        Content.append('None')
        continue
        
        
driver.close()

# major_df = pd.DataFrame({'Title': Title, 'Company' : Company, 'Content' : Content, 'Link' : Link, 'Location': Location, 'label' : label})
# major_df.to_csv("major.csv", index = False)

In [12]:
major_df = pd.DataFrame({'Title': Title, 'Content' : Content, 'Link' : Link, 'Location': Location})
major_df.to_csv("major.csv", index = False)

In [13]:
major_ex_df = pd.DataFrame({'Label':label, 'Company':Company})
major_ex_df.to_csv("major_ex.csv", index = False)